# Hands-on with Weaviate: Queries

In [1]:
# # For Colab
# !pip install -U weaviate-client

## Preparation

### Instantiate Weaviate client

In [2]:
import weaviate
import os

client = weaviate.Client(
    "https://edu-demo.weaviate.network",
    auth_client_secret=weaviate.AuthApiKey("learn-weaviate"),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]
    }
)

### Inspect database

In [3]:
schema = client.schema.get()

In [4]:
# What does the schema look like?
schema

{'classes': [{'class': 'WineReview',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-openai': {'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'properties': [{'dataType': ['text'],
     'description': 'Review body',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'review_body',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'Name of the wine',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'title',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'Originating country',
     'indexFiltera

#### Note: Weaviate data structures:

- `class`: A collection of objects (like a SQL table)
- `properties`: Object properties (like a SQL column)

In [5]:
# What classes are in the instance?
c_names = [c["class"] for c in schema["classes"]]
c_names

['WineReview',
 'WikiArticle',
 'WikiCity',
 'JeopardyCategory',
 'JeopardyQuestion',
 'Publication',
 'Author',
 'Article',
 'Category',
 'MultiLingualReview']

In [6]:
# Inspect a particular class
class_schema = client.schema.get('WineReview')
class_schema

{'class': 'WineReview',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'text2vec-openai': {'model': 'ada',
   'modelVersion': '002',
   'type': 'text',
   'vectorizeClassName': True}},
 'properties': [{'dataType': ['text'],
   'description': 'Review body',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'review_body',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'Name of the wine',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'title',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'Originating country',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text

In [7]:
# What properties are in a particular class?
p_names = [p["name"] for p in class_schema["properties"]]
p_names

['review_body', 'title', 'country', 'points', 'price']

## Search - basic

### Fetch items from Weaviate

In [8]:
# Basic Get query
# Let's use the property names from above

response = (
    client.query
    .get("WineReview", p_names)
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'WineReview': [{'country': 'US',
     'points': 90,
     'price': 32,
     'review_body': "Full bodied and softly tannic, this wine's fruit is front and center, wrapped in a subtle jacket of sweet oak. It dazzles with blackberries, cherries, raspberries, currants and chocolate flavors. Drink now for its sheer lusciousness.",
     'title': 'White Oak 2009 Cabernet Sauvignon (Napa Valley)'},
    {'country': 'France',
     'points': 85,
     'price': 15,
     'review_body': 'Firm tannins make this wine solid and dry. It has banana and bright cherry fruits well integrated with the acidity. The aftertaste suggests the wine could benefit from a few more months before drinking, so wait until 2017.',
     'title': 'Domaine Foretal 2015  Beaujolais-Villages'},
    {'country': 'France',
     'points': 94,
     'price': 0,
     'review_body': 'This firmly structured, beautifully crafted wine has a solid backdrop of tannins and plenty of concentration. Beyond that, it is already 

**Quiz**: In what order do these objects come from?

### Specify the fetched properties

In [9]:
# Modify properties to be fetched

response = (
    client.query
    .get("WineReview", ["title"])
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'WineReview': [{'title': 'White Oak 2009 Cabernet Sauvignon (Napa Valley)'},
    {'title': 'Domaine Foretal 2015  Beaujolais-Villages'},
    {'title': 'Château Rauzan-Ségla 2012  Margaux'}]}}}

In [10]:
# Raw GraphQL query

gql_query = """
{
    Get {
        WineReview (limit: 3) {
            title
        }
    }
}
"""

gql_response = (
    client.query.raw(gql_query)
)
gql_response

{'data': {'Get': {'WineReview': [{'title': 'White Oak 2009 Cabernet Sauvignon (Napa Valley)'},
    {'title': 'Domaine Foretal 2015  Beaujolais-Villages'},
    {'title': 'Château Rauzan-Ségla 2012  Margaux'}]}}}

### Fetch additional properties

In [11]:
# Fetch object ID / vector
response = (
    client.query
    .get("WineReview", ["title"])
    .with_additional(["id", "vector"])
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'id': '01131ce1-d0be-5380-af48-e3cc5c455a63',
      'vector': [0.024616979,
       -0.035291832,
       -0.0108798845,
       -0.03026526,
       0.010106057,
       -0.001083028,
       -0.006640367,
       -0.009259476,
       -0.020450216,
       -0.027699063,
       -0.017103575,
       0.02127034,
       -0.0064121876,
       -0.01109153,
       -0.010112671,
       0.02534451,
       0.036958538,
       -0.012010864,
       0.0176856,
       -0.009047831,
       -0.016190857,
       -0.007361284,
       -0.010046532,
       -0.0051985346,
       0.016574463,
       0.022923818,
       -0.008492263,
       -0.014180227,
       -0.009471121,
       -0.001992441,
       -0.001580725,
       0.004474311,
       0.019603634,
       -0.0054399422,
       0.018122118,
       -0.00914704,
       -0.02497413,
       -0.024048183,
       0.019220028,
       0.024934446,
       0.008465807,
       0.0014997044,
       -0.0037666229,
       0

#### Fetch cross-referenced properties

Cross-references are like relationships in SQL 

In [12]:
# Fetch JeopardyQuestion item
response = (
    client.query
    .get("JeopardyQuestion", ["question" ,"answer"])
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'JeopardyQuestion': [{'answer': 'Jonah',
     'question': 'This prophet passed the time he spent inside a fish offering up prayers'},
    {'answer': 'lay eggs',
     'question': 'Pythons are oviparous, meaning they do this'},
    {'answer': 'Wal-Mart',
     'question': "In August 1996 this chain, the world's largest retailer, opened its first outlets in China"}]}}}

In [13]:
# Show `JeopardyQuestion` schema
client.schema.get("JeopardyQuestion")

{'class': 'JeopardyQuestion',
 'description': 'A Jeopardy! question',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'text2vec-openai': {'model': 'ada',
   'modelVersion': '002',
   'type': 'text',
   'vectorizeClassName': True}},
 'properties': [{'dataType': ['text'],
   'description': 'Question asked to the contestant',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'question',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'Answer provided by the contestant',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'answer',
   'tokenization': 'word'},
  {'dataType': ['int'],
   'description': 'Points that the questi

In [14]:
# Show x-referenced class schema
client.schema.get("JeopardyCategory")

{'class': 'JeopardyCategory',
 'description': 'A Jeopardy! category',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'text2vec-openai': {'model': 'ada',
   'modelVersion': '002',
   'type': 'text',
   'vectorizeClassName': True}},
 'properties': [{'dataType': ['text'],
   'description': "This property was generated by Weaviate's auto-schema feature on Wed May 31 09:09:55 2023",
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': False}},
   'name': 'title',
   'tokenization': 'word'}],
 'replicationConfig': {'factor': 1},
 'shardingConfig': {'virtualPerPhysical': 128,
  'desiredCount': 1,
  'actualCount': 1,
  'desiredVirtualCount': 128,
  'actualVirtualCount': 128,
  'key': '_id',
  'strategy': 'hash',
  'function': 'murmur3'},
 'vectorIndexConfig': {'skip': False,
  '

In [15]:
# Fetch JeopardyQuestion item
response = (
    client.query
    .get("JeopardyQuestion", ["question" ,"answer", "hasCategory {...on JeopardyCategory {title}}"])
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'JeopardyQuestion': [{'answer': 'Jonah',
     'hasCategory': [{'title': 'THE BIBLE'}],
     'question': 'This prophet passed the time he spent inside a fish offering up prayers'},
    {'answer': 'lay eggs',
     'hasCategory': [{'title': 'ANIMALS'}],
     'question': 'Pythons are oviparous, meaning they do this'},
    {'answer': 'Wal-Mart',
     'hasCategory': [{'title': 'BUSINESS & INDUSTRY'}],
     'question': "In August 1996 this chain, the world's largest retailer, opened its first outlets in China"}]}}}

## Similarity-based searches

### NearText search

In [16]:
# NearText query - (any suggestions?)
response = (
    client.query
    .get("WineReview", ["title", "review_body"])
    .with_limit(3)
    .with_near_text({
        "concepts": [
            "very fancy wine"
        ]
    })
    .do()
)

response

{'data': {'Get': {'WineReview': [{'review_body': 'Rich in leather and oak, with swirls of ripe red raspberry and blackberry cobbler, this is a big-boy Zinfandel with hair on its chest, a taste of vanilla on the finish.',
     'title': 'Seven Deadly Zins 2011 Old Vine Zinfandel (Lodi)'},
    {'review_body': 'Dry, smooth and classy, with rich chocolate, blackberry and cherry pie, anise and oak flavors. A bit too soft and sweet for extended aging, but will provide pleasant drinking over the next six years.',
     'title': 'Clos La Chance 2008 Whitestone Vineyard Cabernet Sauvignon (Central Coast)'},
    {'review_body': "With notions of cherry and cinnamon on the nose and just slight fizz, this is a refreshing, fruit-driven sparkling rosé that's full of strawberry and cherry notes—it might just be the very definition of easy summer wine. It ends dry, yet refreshing.",
     'title': 'Gebeshuber 2013 Frizzante Rosé Pinot Noir (Österreichischer Perlwein)'}]}}}

### NearObject

In [17]:
# NearObject query - first, grab an object ID
response = (
    client.query
    .get("WineReview", ["title", "review_body"])
    .with_limit(3)
    .with_near_text({
        "concepts": [
            "very fancy wine"
        ]
    })
    .with_additional("id")
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'id': 'd924c52c-c845-5c0f-b3c8-22434977ee57'},
     'review_body': 'Rich in leather and oak, with swirls of ripe red raspberry and blackberry cobbler, this is a big-boy Zinfandel with hair on its chest, a taste of vanilla on the finish.',
     'title': 'Seven Deadly Zins 2011 Old Vine Zinfandel (Lodi)'},
    {'_additional': {'id': '14c0caa6-0dd1-5154-b907-3490682979e1'},
     'review_body': 'Dry, smooth and classy, with rich chocolate, blackberry and cherry pie, anise and oak flavors. A bit too soft and sweet for extended aging, but will provide pleasant drinking over the next six years.',
     'title': 'Clos La Chance 2008 Whitestone Vineyard Cabernet Sauvignon (Central Coast)'},
    {'_additional': {'id': 'f6da868f-9044-5b4d-87dd-21e1ffffbbf1'},
     'review_body': "With notions of cherry and cinnamon on the nose and just slight fizz, this is a refreshing, fruit-driven sparkling rosé that's full of strawberry and cherry notes—it might

In [18]:
# NearObject query - use that ID to run a search
response = (
    client.query
    .get("WineReview", ["title", "review_body"])
    .with_limit(3)
    .with_near_object({
        "id": "f6da868f-9044-5b4d-87dd-21e1ffffbbf1"
    })
    .do()
)

response

{'data': {'Get': {'WineReview': [{'review_body': "With notions of cherry and cinnamon on the nose and just slight fizz, this is a refreshing, fruit-driven sparkling rosé that's full of strawberry and cherry notes—it might just be the very definition of easy summer wine. It ends dry, yet refreshing.",
     'title': 'Gebeshuber 2013 Frizzante Rosé Pinot Noir (Österreichischer Perlwein)'},
    {'review_body': 'Beautifully perfumed, with acidity, white fruits and a mineral context. The wine is layered with citrus and lime, hints of fresh pineapple acidity. Screw cap.',
     'title': 'Stadt Krems 2009 Steinterrassen Riesling (Kremstal)'},
    {'review_body': 'This ripe wine with its red fruits and well-balanced texture is beautifully ready to drink, with just the right crisp acidity to balance the soft richness. There is no sign yet of maturity, the fruit from this great vintage showing at its perfumed best. Drink the wine now.',
     'title': 'Pol Roger 2008 Rosé Brut  (Champagne)'}]}}}

### NearVector

In [19]:
# NearVector query - grab a vector

In [20]:
# Grab a vector from OpenAI
import openai
openai.api_key = os.environ["OPENAI_APIKEY"]
resp = openai.Embedding.create(
  model="text-embedding-ada-002",
  input="Argentinian wine that goes well with fish"
)
resp

<OpenAIObject list at 0x109daf600> JSON: {
  "data": [
    {
      "embedding": [
        0.011122459545731544,
        -0.01030720118433237,
        0.003110600635409355,
        -0.010837766341865063,
        -0.0012649452546611428,
        0.009090783074498177,
        -0.03305033594369888,
        -0.011640084907412529,
        -0.0016871328698471189,
        -0.043247539550065994,
        -0.0018650663550943136,
        0.012966497801244259,
        0.01715925708413124,
        -0.009750754572451115,
        -0.021274372935295105,
        0.024095427244901657,
        0.03082454949617386,
        -0.004354517441242933,
        0.0394430011510849,
        -0.017780406400561333,
        -0.02214139513671398,
        0.0076802559196949005,
        0.039287712424993515,
        -0.012539457529783249,
        0.0067032393999397755,
        0.00911666452884674,
        -0.011433035135269165,
        -0.01111598964780569,
        -0.0031300117261707783,
        -0.028650525957345963,
   

In [21]:
emb = resp["data"][0]["embedding"]
len(emb)

1536

In [22]:
# NearVector query - use the vector to run a search
response = (
    client.query
    .get("WineReview", ["title", "review_body", "country"])
    .with_limit(3)
    .with_near_vector({
        "vector": emb
    })
    .do()
)

response

{'data': {'Get': {'WineReview': [{'country': 'Argentina',
     'review_body': 'Generous, sizable aromas of cassis, blackberry and violet are inviting. This is juicy, with slightly wiry acidity creating a racy mouthfeel. Flavors of berry and cassis are loamy on their own but ride comfortably on a wave of tartaric acidity, while the finish is lifted and lively, with black-fruit flavors.',
     'title': 'Doña Silvina 2011 Malbec (Mendoza)'},
    {'country': 'Argentina',
     'review_body': 'Vegetal-leaning, stewy aromas of plum and jammy berry are saucy. This feels sticky and low in acidity. Grassy, herbal plum and berry flavors are a bit green on the finish.',
     'title': 'Finca Flichman 2015 Misterio Malbec (Mendoza)'},
    {'country': 'Argentina',
     'review_body': "Fruity on the nose, with a friendly mix of pineapple, apple, melon and powdered sugar aromas. Feels smooth and round but not heavy, with good acidity and likable, simple flavors of melon, banana and apple. There's a lig

**Discussion**: What's going on under the hood?

### Get distances to results

In [23]:
# Fetch distances in the results
response = (
    client.query
    .get("WineReview", ["title", "review_body", "country"])
    .with_limit(3)
    .with_near_vector({
        "vector": emb
    })
    .with_additional("distance")
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'distance': 0.13938928},
     'country': 'Argentina',
     'review_body': 'Generous, sizable aromas of cassis, blackberry and violet are inviting. This is juicy, with slightly wiry acidity creating a racy mouthfeel. Flavors of berry and cassis are loamy on their own but ride comfortably on a wave of tartaric acidity, while the finish is lifted and lively, with black-fruit flavors.',
     'title': 'Doña Silvina 2011 Malbec (Mendoza)'},
    {'_additional': {'distance': 0.14162123},
     'country': 'Argentina',
     'review_body': 'Vegetal-leaning, stewy aromas of plum and jammy berry are saucy. This feels sticky and low in acidity. Grassy, herbal plum and berry flavors are a bit green on the finish.',
     'title': 'Finca Flichman 2015 Misterio Malbec (Mendoza)'},
    {'_additional': {'distance': 0.14296353},
     'country': 'Argentina',
     'review_body': "Fruity on the nose, with a friendly mix of pineapple, apple, melon and powdered s

### Modify thresholds

In [24]:
# Add a distance threshold
response = (
    client.query
    .get("WineReview", ["title", "review_body", "country"])
    .with_near_vector({
        "vector": emb,
        "distance": 0.145
    })
    .with_additional("distance")
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'distance': 0.13938928},
     'country': 'Argentina',
     'review_body': 'Generous, sizable aromas of cassis, blackberry and violet are inviting. This is juicy, with slightly wiry acidity creating a racy mouthfeel. Flavors of berry and cassis are loamy on their own but ride comfortably on a wave of tartaric acidity, while the finish is lifted and lively, with black-fruit flavors.',
     'title': 'Doña Silvina 2011 Malbec (Mendoza)'},
    {'_additional': {'distance': 0.14162123},
     'country': 'Argentina',
     'review_body': 'Vegetal-leaning, stewy aromas of plum and jammy berry are saucy. This feels sticky and low in acidity. Grassy, herbal plum and berry flavors are a bit green on the finish.',
     'title': 'Finca Flichman 2015 Misterio Malbec (Mendoza)'},
    {'_additional': {'distance': 0.14296353},
     'country': 'Argentina',
     'review_body': "Fruity on the nose, with a friendly mix of pineapple, apple, melon and powdered s

**Discussion**: Why do we need thresholds / limits?

## Keyword (BM25) searches

In [25]:
# Try a keyword search for "apple"
response = (
    client.query.get("WineReview", ["title", "review_body", "country"])
    .with_bm25("apple")
    .with_additional("score")
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'score': '0.9366371'},
     'country': 'Argentina',
     'review_body': "Fruity on the nose, with a friendly mix of pineapple, apple, melon and powdered sugar aromas. Feels smooth and round but not heavy, with good acidity and likable, simple flavors of melon, banana and apple. There's a light note of toast on the finish.",
     'title': 'Alamos 2010 Chardonnay (Mendoza)'},
    {'_additional': {'score': '0.85285914'},
     'country': 'Argentina',
     'review_body': "This warm-climate Chardonnay displays aromas of baked apple, toast and spice. It's resiny and heavy in the mouth, but there's enough acidity to maintain balance. Flavors of honeyed peach, baked apple, cinnamon and white pepper finish in tropical fashion, where banana is the lasting fruit note.",
     'title': 'Viña Cobos 2012 Bramare Marchiori Vineyard Chardonnay (Perdriel)'},
    {'_additional': {'score': '0.6943899'},
     'country': 'Portugal',
     'review_body': 'A sof

**Discussion**: How do keyword (BM25) searches work?

## Hybrid searches

In [26]:
# Try a hybrid search for "white wine easy drink"
response = (
    client.query.get("WineReview", ["title", "review_body", "country"])
    .with_hybrid("white wine easy drink")
    .with_additional("score")
    .with_limit(3)
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'score': '0.016263336'},
     'country': 'Austria',
     'review_body': "With notions of cherry and cinnamon on the nose and just slight fizz, this is a refreshing, fruit-driven sparkling rosé that's full of strawberry and cherry notes—it might just be the very definition of easy summer wine. It ends dry, yet refreshing.",
     'title': 'Gebeshuber 2013 Frizzante Rosé Pinot Noir (Österreichischer Perlwein)'},
    {'_additional': {'score': '0.016129032'},
     'country': 'Portugal',
     'review_body': 'In a medium sweet style, the wine has a light prickle and fine citrus acidity to balance the soft freshness. It is a refreshing wine, totally drinkable immediately.',
     'title': 'Messias 2015 Santola White (Vinho Verde)'},
    {'_additional': {'score': '0.011904762'},
     'country': 'US',
     'review_body': 'Dry, smooth and classy, with rich chocolate, blackberry and cherry pie, anise and oak flavors. A bit too soft and sweet for ext

**Discussion**: How do hybrid searches work?

## Conditional (`where`) Filters

### Single filters

In [27]:
# Single filter with price
response = (
    client.query
    .get("WineReview", ["title", "review_body", "country"])
    .with_limit(3)
    .with_near_vector({
        "vector": emb
    })
    .with_where({
        "path": ["price"],
        "operator": "GreaterThan",
        "valueNumber": 10
    })
    .do()
)

response

{'data': {'Get': {'WineReview': [{'country': 'Argentina',
     'review_body': 'Generous, sizable aromas of cassis, blackberry and violet are inviting. This is juicy, with slightly wiry acidity creating a racy mouthfeel. Flavors of berry and cassis are loamy on their own but ride comfortably on a wave of tartaric acidity, while the finish is lifted and lively, with black-fruit flavors.',
     'title': 'Doña Silvina 2011 Malbec (Mendoza)'},
    {'country': 'Argentina',
     'review_body': "Fruity on the nose, with a friendly mix of pineapple, apple, melon and powdered sugar aromas. Feels smooth and round but not heavy, with good acidity and likable, simple flavors of melon, banana and apple. There's a light note of toast on the finish.",
     'title': 'Alamos 2010 Chardonnay (Mendoza)'},
    {'country': 'Argentina',
     'review_body': "This warm-climate Chardonnay displays aromas of baked apple, toast and spice. It's resiny and heavy in the mouth, but there's enough acidity to maintain 

#### Filter by partial matches

In [28]:
# Filter with partial strin
response = (
    client.query
    .get("WineReview", ["title", "review_body", "country"])
    .with_limit(3)
    .with_near_vector({
        "vector": emb
    })
    .with_where({
        "path": ["review_body"],
        "operator": "Like",
        "valueText": "*citrus*"
    })
    .do()
)

response

{'data': {'Get': {'WineReview': [{'country': 'US',
     'review_body': 'There is a delicious, citrusy kick to this wine, lifting the bright berry fruit flavors. The flavors pile on, with peppery spice and chocolate, and a complex midpalate that tastes like cherry cola.',
     'title': 'Soléna 2009 Guadalupe Vineyard Pinot Noir'},
    {'country': 'Portugal',
     'review_body': 'In a medium sweet style, the wine has a light prickle and fine citrus acidity to balance the soft freshness. It is a refreshing wine, totally drinkable immediately.',
     'title': 'Messias 2015 Santola White (Vinho Verde)'},
    {'country': 'Italy',
     'review_body': 'Aromas of toasted nut and dried stone fruit lead the nose on this structured white. On the palate, dried herb and a mineral note accent the core of yellow apple and citrus. Fresh acidity generates a clean finish.',
     'title': 'Borgo San Daniele 2013 Friulano (Friuli Isonzo)'}]}}}

#### Filter by cross-references

In [29]:
# Filter with cross-ref property (look for *history*)
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer", "hasCategory {...on JeopardyCategory {title}}"])
    .with_limit(5)
    .with_where({
        "path": ["hasCategory", "JeopardyCategory", "title"],
        "operator": "Like",
        "valueText": "*history*"
    })
    .do()
)

response

{'data': {'Get': {'JeopardyQuestion': [{'answer': 'Anti-Masonic party',
     'hasCategory': [{'title': 'AMERICAN HISTORY'}],
     'question': '1st to hold a nominating convention, this 3rd party ironically chose Mason Wm. Wirt to run for Pres.'},
    {'answer': 'Hannibal',
     'hasCategory': [{'title': 'HISTORY'}],
     'question': 'In 218 B.C. this Carthaginian defeated the Romans at the Trebia River'},
    {'answer': 'Francisco Franco',
     'hasCategory': [{'title': 'HISTORY'}],
     'question': 'In October 1936 he was named "El Caudillo" of nationalist Spain'},
    {'answer': 'not guilty by reason of insanity',
     'hasCategory': [{'title': 'U.S. HISTORY'}],
     'question': 'The verdict in the 1982 trial of John Hinckley Jr. for attempting to murder the president'},
    {'answer': 'Franklin D. Roosevelt',
     'hasCategory': [{'title': 'U.S. HISTORY'}],
     'question': '8 days after his March 4, 1933 inauguration, he gave his first Fireside Chat'}]}}}

In [30]:
# Like *history* vs Like *histor*

### Nested filters

In [31]:
# Filter with nested filters
# `points` greater than 1000 or Like *histor*
where_filter = {
    "operator": "Or",
    "operands": [
        {
            "path": ["hasCategory", "JeopardyCategory", "title"],
            "operator": "Like",
            "valueText": "*histor*"
        },   
        {
            "path": ["points"],
            "operator": "GreaterThan",
            "valueInt": 1000
        },
    ]
}


response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer", "points", "hasCategory {...on JeopardyCategory {title}}"])
    .with_limit(5)
    .with_where(where_filter)
    .do()
)

response

{'data': {'Get': {'JeopardyQuestion': [{'answer': 'Baffin Bay',
     'hasCategory': [{'title': 'BODIES OF WATER'}],
     'points': 2000,
     'question': 'For most of the year, ice floes & icebergs prevent navigation in this bay between Greenland & NE Canada'},
    {'answer': 'Anti-Masonic party',
     'hasCategory': [{'title': 'AMERICAN HISTORY'}],
     'points': 800,
     'question': '1st to hold a nominating convention, this 3rd party ironically chose Mason Wm. Wirt to run for Pres.'},
    {'answer': 'West Virginia',
     'hasCategory': [{'title': 'AMERICANA'}],
     'points': 2000,
     'question': "Jay Rockefeller's rich life includes serving as a U.S. senator from this state since 1985"},
    {'answer': 'Hannibal',
     'hasCategory': [{'title': 'HISTORY'}],
     'points': 600,
     'question': 'In 218 B.C. this Carthaginian defeated the Romans at the Trebia River'},
    {'answer': 'Metamorphoses',
     'hasCategory': [{'title': 'LITERATURE'}],
     'points': 1600,
     'question

## Generative searches

In [32]:
# NearText with Generative
response = (
    client.query
    .get("WikiArticle", ["title"])
    .with_limit(1)
    .with_near_text({
        "concepts": [
            "australia"
        ]
    })
    .with_generate(
        single_prompt="Summarize this article {wiki_summary}"
    )
    .do()
)

response

{'data': {'Get': {'WikiArticle': [{'_additional': {'generate': {'error': None,
       'singleResult': 'The Sydney Opera House is a famous and distinctive multi-venue performing arts centre located on the foreshore of Sydney Harbour. Designed by Danish architect Jørn Utzon, it was completed by an Australian team and formally opened in 1973. The building comprises multiple performance venues and hosts over 1,500 performances annually, attended by more than 1.2 million people. It is managed by the Sydney Opera House Trust and is visited by over eight million people annually. In 2007, it became a UNESCO World Heritage Site and has been listed on various other heritage registers.'}},
     'title': 'Sydney Opera House'}]}}}

In [33]:
# Get grouped data
response = (
    client.query
    .get("WineReview", ["title"])
    .with_limit(10)
    .with_near_text({
        "concepts": [
            "dessert wine"
        ]
    })
    .with_generate(
        grouped_task="Based on these reviews, what should you look for in a dessert wine? Provide three bullet points"
    )
    .do()
)

response

{'data': {'Get': {'WineReview': [{'_additional': {'generate': {'error': None,
       'groupedResult': '1. Look for a wine with rich, fruity flavors such as blackberry, cherry, and strawberry.\n2. Consider the level of sweetness, as some dessert wines may be too soft and sweet for extended aging.\n3. Pay attention to the acidity and tannins, which can balance out the sweetness and add complexity to the wine.'}},
     'title': 'Clos La Chance 2008 Whitestone Vineyard Cabernet Sauvignon (Central Coast)'},
    {'_additional': {'generate': None},
     'title': 'Dynasty Cellars 2007 DC3 Meritage (Walla Walla Valley (WA))'},
    {'_additional': {'generate': None},
     'title': 'Gebeshuber 2013 Frizzante Rosé Pinot Noir (Österreichischer Perlwein)'},
    {'_additional': {'generate': None},
     'title': 'Schmitz 24 Brix 2012 Sauvignon Blanc (Sierra Foothills)'},
    {'_additional': {'generate': None},
     'title': 'Château Saint-Sernin 2007 Malbec (Cahors)'},
    {'_additional': {'generate':